In [13]:
%run "Ch0 - setup.ipynb"

In [7]:
import polars as pl

url = "https://theunitedstates.io/congress-legislators/legislators-historical.csv"

dtypes = {
    "first_name": pl.Categorical,
    "gender": pl.Categorical,
    "type": pl.Categorical,
    "state": pl.Categorical,
    "party": pl.Categorical,
}

dataset = pl.read_csv(url, dtypes=dtypes).with_columns(pl.col("birthday").str.strptime(pl.Date, strict=False))
print(dataset)

shape: (12136, 36)
┌──────────┬──────────┬───────────┬────────┬─────┬────────────┬────────────┬──────────┬────────────┐
│ last_nam ┆ first_na ┆ middle_na ┆ suffix ┆ ... ┆ ballotpedi ┆ washington ┆ icpsr_id ┆ wikipedia_ │
│ e        ┆ me       ┆ me        ┆ ---    ┆     ┆ a_id       ┆ _post_id   ┆ ---      ┆ id         │
│ ---      ┆ ---      ┆ ---       ┆ str    ┆     ┆ ---        ┆ ---        ┆ i64      ┆ ---        │
│ str      ┆ cat      ┆ str       ┆        ┆     ┆ str        ┆ str        ┆          ┆ str        │
╞══════════╪══════════╪═══════════╪════════╪═════╪════════════╪════════════╪══════════╪════════════╡
│ Bassett  ┆ Richard  ┆ null      ┆ null   ┆ ... ┆ null       ┆ null       ┆ 507      ┆ Richard    │
│          ┆          ┆           ┆        ┆     ┆            ┆            ┆          ┆ Bassett    │
│          ┆          ┆           ┆        ┆     ┆            ┆            ┆          ┆ (Delaware  │
│          ┆          ┆           ┆        ┆     ┆            ┆         

In [8]:
q = (
    dataset.lazy()
    .groupby("first_name")
    .agg(
        [
            pl.count(),
            pl.col("gender"),
            pl.first("last_name"),
        ]
    )
    .sort("count", descending=True)
    .limit(5)
)

df = q.collect()
print(df)

shape: (5, 4)
┌────────────┬───────┬─────────────────────┬───────────┐
│ first_name ┆ count ┆ gender              ┆ last_name │
│ ---        ┆ ---   ┆ ---                 ┆ ---       │
│ cat        ┆ u32   ┆ list[cat]           ┆ str       │
╞════════════╪═══════╪═════════════════════╪═══════════╡
│ John       ┆ 1256  ┆ ["M", "M", ... "M"] ┆ Walker    │
│ William    ┆ 1022  ┆ ["M", "M", ... "M"] ┆ Few       │
│ James      ┆ 714   ┆ ["M", "M", ... "M"] ┆ Armstrong │
│ Thomas     ┆ 454   ┆ ["M", "M", ... "M"] ┆ Tucker    │
│ Charles    ┆ 439   ┆ ["M", "M", ... "M"] ┆ Carroll   │
└────────────┴───────┴─────────────────────┴───────────┘


In [10]:
q = (
    dataset.lazy()
    .groupby("state")
    .agg(
        [
            (pl.col("party") == "Anti-Administration").sum().alias("anti"),
            (pl.col("party") == "Pro-Administration").sum().alias("pro"),
        ]
    )
    .sort("pro", descending=True)
    .limit(5)
)

df = q.collect()
df

state,anti,pro
cat,u32,u32
"""NJ""",0,3
"""CT""",0,3
"""NC""",1,2
"""MA""",0,1
"""VA""",3,1


In [14]:
def compute_age() -> pl.Expr:
    return date(2021, 1, 1).year - pl.col("birthday").dt.year()


def avg_birthday(gender: str) -> pl.Expr:
    return compute_age().filter(pl.col("gender") == gender).mean().alias(f"avg {gender} birthday")


q = (
    dataset.lazy()
    .groupby(["state"])
    .agg(
        [
            avg_birthday("M"),
            avg_birthday("F"),
            (pl.col("gender") == "M").sum().alias("# male"),
            (pl.col("gender") == "F").sum().alias("# female"),
        ]
    )
    .limit(5)
)

df = q.collect()
df

state,avg M birthday,avg F birthday,# male,# female
cat,f64,f64,u32,u32
"""PA""",179.724846,91.857143,1050,7
"""TX""",131.637405,78.833333,263,6
"""NM""",139.0,67.666667,52,6
"""SD""",141.085106,91.0,47,4
"""KY""",183.360704,103.0,372,2


In [33]:
def get_person() -> pl.Expr:
    return pl.col("first_name") + pl.lit(" ") + pl.col("last_name")


q = (
    dataset.lazy()
    .sort("birthday", descending=True)
    .groupby(["state"])
    .agg(
        [
            get_person().first().alias("youngest"),
            get_person().last().alias("oldest"),
        ]
    ).sort("state")
    .limit(5)
)

df = q.collect()
df


state,youngest,oldest
cat,str,str
"""DE""","""John Carney""","""Samuel White"""
"""VA""","""Scott Taylor""","""William Grayso..."
"""SC""","""Joe Cunningham...","""Ralph Izard"""
"""MD""","""Frank Kratovil...","""Benjamin Conte..."
"""PA""","""Conor Lamb""","""Thomas Fitzsim..."


In [86]:
def get_person() -> pl.Expr:
    return pl.col("first_name") + pl.lit(" ") + pl.col("last_name")


q = (
    dataset.lazy()
    .sort("birthday", descending=True)
    .groupby(["state"])
    .agg(
        [
            get_person().first().alias("youngest"),
            get_person().last().alias("oldest"),
            get_person().sort().first().alias("alphabetical_first"),
        ]
    ).sort("state")
    .limit(5)
)

df = q.collect()
df

state,youngest,oldest,alphabetical_first
cat,str,str,str
"""DE""","""John Carney""","""Samuel White""","""Albert Polk"""
"""VA""","""Scott Taylor""","""William Grayso...","""A. McEachin"""
"""SC""","""Joe Cunningham...","""Ralph Izard""","""Abraham Nott"""
"""MD""","""Frank Kratovil...","""Benjamin Conte...","""Albert Blakene..."
"""PA""","""Conor Lamb""","""Thomas Fitzsim...","""Aaron Kreider"""


In [101]:
def get_person() -> pl.Expr:
    return pl.col("first_name") + pl.lit(" ") + pl.col("last_name")


q = (
    dataset.lazy()
    .sort("birthday", descending=True)
    .groupby(["state"])
    .agg(
        [
            get_person().first().alias("youngest"),
            get_person().last().alias("oldest"),
            get_person().sort().first().alias("alphabetical_first"),
            pl.col("gender").sort_by("first_name").first().alias("gender"),
        ]
    )
    .sort("state").filter((pl.col("gender") == 'M') & (pl.col("state") == 'DE')).limit(5)
)

df = q.collect()
df


state,youngest,oldest,alphabetical_first,gender
cat,str,str,str,cat
"""DE""","""John Carney""","""Samuel White""","""Albert Polk""","""M"""
